# 📊 Proyecto: Análisis y visualización de la calidad del aire: una exploración de contaminantes atmosféricos y su relación con el PM2.5
**Curso:** Samsung Innovation Campus – Módulo de Python (Ecuador 2025)  
**Seccion:** EC03  

---

## 🧩 Módulo 1: Adquisición de Datos

En esta sección se carga el dataset, se explora su estructura general y se validan los rangos de los contaminantes.

El dataset contiene mediciones de diferentes gases y partículas contaminantes en el aire, registradas con fecha y hora.


### 📘 Descripción general de las columnas del dataset

| Columna | Descripción | Relevancia |
|----------|--------------|-------------|
| 📅 **Date** | Fecha y hora del registro | Permite analizar variaciones temporales. |
| 🏭 **CO** | Monóxido de carbono (ppm) | Gas tóxico, indicador de combustión incompleta. |
| 🚗 **NO** | Óxido nítrico | Se genera en la quema de combustibles fósiles. |
| 🚙 **NO2** | Dióxido de nitrógeno | Contaminante urbano, precursor del ozono. |
| 🌫️ **O3** | Ozono troposférico | Se forma con NOx + luz solar; irritante respiratorio. |
| 🌋 **SO2** | Dióxido de azufre | Proviene de la quema de carbón y petróleo. |
| ☁️ **PM2.5** | Partículas finas < 2.5 μm | Altamente dañinas, ingresan a los pulmones. |
| 🌧️ **PM10** | Partículas < 10 μm | Menos dañinas, quedan en vías respiratorias. |
| 🍃 **NH3** | Amoníaco | De origen agrícola, contribuye a partículas secundarias. |


In [73]:
# ===== Importar librerías principales ====
import pandas as pd
import numpy as np
import warnings
import importlib
import src.data_acquisition as da
import src.data_processing as dp
import src.data_visualization as dv
import src.data_interpretation as di

# Recargar los módulos para reflejar cambios recientes
importlib.reload(dp) 
importlib.reload(da)
importlib.reload(dv)
importlib.reload(di)

warnings.filterwarnings('ignore')

In [76]:
# === Listar los archivos disponibles en la carpeta de datos ===
print("="*60)
print("📂 Archivos disponibles en la carpeta")
print("="*60)
for file in da.list_data_files("data"):
    print(f"- {file}")

📂 Archivos disponibles en la carpeta
- data\delhiaqi.csv


In [83]:
# Cargar de los datos del archivo CSV en un DataFrame
df_air_quality = da.load_data_csv('data\delhiaqi.csv')

✅ Archivo 'data\delhiaqi.csv' cargado exitosamente.


In [ ]:
print("=" * 60)
print("📊 Exploración inicial de los datos")
print("=" * 60)


📊 Exploración inicial de los datos
✅ Archivo 'data\delhiaqi.csv' cargado exitosamente.
